# The Problem

We will create a model based on simple artifial neural networks (ANNs) each hand image to the corresponding number the hand is depicting ($0$ to $5$).

# Data Ingest

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [12]:
def construct_dataset_file(directory):
    df = pd.DataFrame({
        'label': pd.Series(dtype='int'), 
        'image': pd.Series(dtype='str')
    })
    for filename in os.listdir(directory):
        file_location = os.path.join(directory, filename)
        image = tf.keras.utils.load_img(file_location, color_mode='grayscale')
        pixels = np.array(image)
        
        label = int(filename[-6])
        df.loc[len(df.index)] = [label, pixels]
        
    return df

def load_train_df(forced=False):
    TRAIN_IMAGES_DIRECTORY = '/kaggle/input/fingers/train'
    TRAIN_DATASET_FILE = '/kaggle/working/train.csv'

    if not os.path.exists(TRAIN_DATASET_FILE) or forced:
        print("Constructing train dataset")
        df = construct_dataset_file(TRAIN_IMAGES_DIRECTORY)
        df.to_csv(TRAIN_DATASET_FILE, index=False)
        return df
    else:
        print("Loading cached train dataset")
        return pd.read_csv(TRAIN_DATASET_FILE)

def load_test_df(forced=False):
    TEST_IMAGES_DIRECTORY = '/kaggle/input/fingers/test'
    TEST_DATASET_FILE = '/kaggle/working/test.csv'

    if not os.path.exists(TEST_DATASET_FILE) or forced:
        print("Constructing test dataset")
        df = construct_dataset_file(TEST_IMAGES_DIRECTORY)
        df.to_csv(TEST_DATASET_FILE, index=False)
        return df
    else:
        print("Loading cached test dataset")
        return pd.read_csv(TEST_DATASET_FILE)
    
# raw_train_df = load_train_df(forced=True)
# raw_test_df = load_test_df(forced=True)
raw_train_df, raw_test_df = load_train_df(), load_test_df()

Constructing train dataset


In [14]:
# Let's have a look at the raw train dataset.
raw_train_df.head()

,label,image
0,4,"[[58, 56, 54, 54, 54, 53, 49, 48, 51, 53, 55, ..."
1,4,"[[52, 50, 47, 44, 41, 41, 43, 43, 41, 39, 37, ..."
2,0,"[[37, 35, 33, 33, 36, 36, 33, 34, 40, 41, 39, ..."
3,4,"[[51, 52, 53, 53, 52, 50, 49, 50, 54, 55, 53, ..."
4,1,"[[67, 66, 65, 66, 68, 70, 72, 72, 71, 68, 63, ..."


In [11]:
# Let's have a look at the raw test dataset.
raw_test_df.head()

,label,image
0,1,[[67 66 64 ... 59 64 66]\n [66 65 64 ... 58 62...
1,0,[[78 75 71 ... 61 56 53]\n [75 74 71 ... 61 57...
2,0,[[58 56 53 ... 58 55 53]\n [59 58 55 ... 59 56...
3,3,[[51 51 53 ... 61 63 64]\n [51 52 54 ... 59 61...
4,4,[[34 34 36 ... 44 42 40]\n [35 35 37 ... 44 40...


In [13]:
def split_X_y(df):
    y = df['filename'].apply(lambda x: x.filename[-6])
    return y

split_X_y(raw_train_df)

KeyError: 'filename'